In [170]:
import pandas as pd
import numpy as np
import requests
import re
from io import StringIO

In [171]:
url = "https://apisidra.ibge.gov.br/values//t/9542/p/2022/v/950/c59/allxt/c2/allxt/c86/allxt/c287/93086,93087,2999,9482,9483,9484/n3/all/h/y/f/n"
r = requests.get(url)
if r.ok:
    df = pd.read_json(StringIO(r.text))
else:
    print('Tente novamente. Status code:', r.status_code)

In [172]:
data = df.rename(columns=df.iloc[0]).drop(df.index[0])
data = data.reset_index(drop=True)
data = data.drop(columns=['Nível Territorial','Unidade de Medida','Variável'])
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3240 entries, 0 to 3239
Data columns (total 7 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Valor                 3240 non-null   object
 1   Ano                   3240 non-null   object
 2   Alfabetização         3240 non-null   object
 3   Sexo                  3240 non-null   object
 4   Cor ou raça           3240 non-null   object
 5   Idade                 3240 non-null   object
 6   Unidade da Federação  3240 non-null   object
dtypes: object(7)
memory usage: 177.3+ KB


In [173]:
negativos = len(data[data['Valor']=='-'])
print(f'Valores negativos:', negativos)
print('---------Cleaning-------')
data['Valor'] = data['Valor'].replace({'-': np.nan})
data['Valor'] = pd.to_numeric(data['Valor'])
data['Ano'] = pd.to_numeric(data['Ano'])
data = data.dropna(subset=['Valor'])
print(data.isnull().sum())
print('-------dtypes-----------')
print(data.dtypes)
data.loc[:, 'Idade'] = data['Idade'].str.replace(r'(\d+)\s*a\s*(\d+)\s*anos', r'\1-\2', regex=True)

Valores negativos: 15
---------Cleaning-------
Valor                   0
Ano                     0
Alfabetização           0
Sexo                    0
Cor ou raça             0
Idade                   0
Unidade da Federação    0
dtype: int64
-------dtypes-----------
Valor                   float64
Ano                       int64
Alfabetização            object
Sexo                     object
Cor ou raça              object
Idade                    object
Unidade da Federação     object
dtype: object


In [174]:
lista = data.columns[1:].tolist()
for coluna in lista:
    print(f'Coluna {coluna}:')
    print(data[coluna].unique())

Coluna Ano:
[2022]
Coluna Alfabetização:
['Alfabetizadas' 'Não alfabetizadas']
Coluna Sexo:
['Homens' 'Mulheres']
Coluna Cor ou raça:
['Branca' 'Preta' 'Amarela' 'Parda' 'Indígena']
Coluna Idade:
['15-19' '20-24' '25-34' '35-44' '45-54' '55-64']
Coluna Unidade da Federação:
['Rondônia' 'Acre' 'Amazonas' 'Roraima' 'Pará' 'Amapá' 'Tocantins'
 'Maranhão' 'Piauí' 'Ceará' 'Rio Grande do Norte' 'Paraíba' 'Pernambuco'
 'Alagoas' 'Sergipe' 'Bahia' 'Minas Gerais' 'Espírito Santo'
 'Rio de Janeiro' 'São Paulo' 'Paraná' 'Santa Catarina'
 'Rio Grande do Sul' 'Mato Grosso do Sul' 'Mato Grosso' 'Goiás'
 'Distrito Federal']


In [175]:
uf = {
    'Rondônia': 'RO',
    'Acre': 'AC',
    'Amazonas': 'AM',
    'Roraima': 'RR',
    'Pará': 'PA',
    'Amapá': 'AP',
    'Tocantins': 'TO',
    'Maranhão': 'MA',
    'Piauí': 'PI',
    'Ceará': 'CE',
    'Rio Grande do Norte': 'RN',
    'Paraíba': 'PB',
    'Pernambuco': 'PE',
    'Alagoas': 'AL',
    'Sergipe': 'SE',
    'Bahia': 'BA',
    'Minas Gerais': 'MG',
    'Espírito Santo': 'ES',
    'Rio de Janeiro': 'RJ',
    'São Paulo': 'SP',
    'Paraná': 'PR',
    'Santa Catarina': 'SC',
    'Rio Grande do Sul': 'RS',
    'Mato Grosso do Sul': 'MS',
    'Mato Grosso': 'MT',
    'Goiás': 'GO',
    'Distrito Federal': 'DF'
}

# Changing the values for its acronyms
cor = {'Branca':'White','Preta':'Black','Amarela':'Asian','Parda':'Latino','Indígena':'Indigenous'}
genero = {'Homens':'Male','Mulheres':'Female'}
alfabetizacao = {'Alfabetizadas':'Yes','Não alfabetizadas':'No'}

data.loc[:, 'Cor ou raça'] = data['Cor ou raça'].replace(cor)
data.loc[:, 'Alfabetização'] = data['Alfabetização'].replace(alfabetizacao)
data.loc[:, 'Unidade da Federação'] = data['Unidade da Federação'].replace(uf)
data.loc[:, 'Sexo'] = data['Sexo'].replace(genero)

# renaming the columns
data = data.rename(columns={"Valor": "Population", "Ano": "Year", 'Alfabetização': 'Literate', 'Cor ou raça': 'Race', 'Idade': 'Age-Group', 'Unidade da Federação': 'State', 'Sexo': 'Gender'})

In [176]:
# exporting
# data.to_csv("/home/satire/PycharmProjects/Statistics/csv/data.csv", index=False)

In [177]:
data

,Population,Year,Literate,Gender,Race,Age-Group,State
0,16855.0,2022,Yes,Male,White,15-19,RO
1,8208.0,2022,Yes,Male,White,15-19,AC
2,30666.0,2022,Yes,Male,White,15-19,AM
3,5093.0,2022,Yes,Male,White,15-19,RR
4,64688.0,2022,Yes,Male,White,15-19,PA
...,...,...,...,...,...,...,...
3235,341.0,2022,No,Female,Indigenous,55-64,RS
3236,844.0,2022,No,Female,Indigenous,55-64,MS
3237,399.0,2022,No,Female,Indigenous,55-64,MT
3238,101.0,2022,No,Female,Indigenous,55-64,GO
